In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Configurations matching docker
DB_CONFIG = {
    'user': os.getenv('DB_USER', 'egg_user'),
    'password': os.getenv('DB_PASSWORD', 'egg_pass'),
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', 5432),
    'database': os.getenv('DB_NAME', 'egg_prices')
}

# Clean connection string
conn_string = (
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

engine = create_engine(conn_string)
df = pd.read_sql("SELECT * FROM economic_data", engine)
df.to_csv('data/egg_price_drivers.csv')